In [ ]:
import xml.etree.ElementTree as ET
import csv

xml_file = 'data_aqi_cpcb.xml'
csv_file = 'converted_aqi.csv'

tree = ET.parse(xml_file)
root = tree.getroot()

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
        "Country", "State", "City", "Station", "Latitude", "Longitude", "Last Update",
        "Pollutant", "Min", "Max", "Avg", "AQI", "Predominant Parameter"
    ])

    for country in root.findall('Country'):
        country_id = country.get('id', '')
        for state in country.findall('State'):
            state_id = state.get('id', '')
            for city in state.findall('City'):
                city_id = city.get('id', '')
                for station in city.findall('Station'):
                    station_id = station.get('id', '')
                    lat = station.get('latitude', '')
                    lon = station.get('longitude', '')
                    last_update = station.get('lastupdate', '')
                    aqi_tag = station.find('Air_Quality_Index')
                    aqi = aqi_tag.get('Value') if aqi_tag is not None else ''
                    param = aqi_tag.get('Predominant_Parameter') if aqi_tag is not None else ''


                    for pollutant in station.findall('Pollutant_Index'):
                        pol_id = pollutant.get('id', '')
                        min_val = pollutant.get('Min', '')
                        max_val = pollutant.get('Max', '')
                        avg_val = pollutant.get('Avg', '')

                        writer.writerow([
                            country_id, state_id, city_id, station_id, lat, lon,
                            last_update, pol_id, min_val, max_val, avg_val,
                            aqi, param
                        ])

print("XML converted to CSV successfully!")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
import xgboost as xg
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv("converted_aqi.csv")
print(df.head())
print(df.isnull().sum())
print(df.info())
df.head

In [ ]:
df['Min'].fillna(df['Min'].mean(), inplace=True)
df['Max'].fillna(df['Max'].mean(), inplace=True)
df['Avg'].fillna(df['Avg'].mean(), inplace=True)
df['AQI'].fillna(df['AQI'].mean(), inplace=True)
df['Predominant Parameter'].fillna('Unknown', inplace=True)


In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.describe())
print(df['Country'].value_counts())
print(df['State'].value_counts())
print(df['City'].value_counts())
print(df['Pollutant'].value_counts())
df.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['AQI'], bins=30, kde=True,color='grey')
plt.title('Distribution of AQI values')
plt.xlabel('AQI')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(x='State', y='AQI', data=df)
plt.title('Box Plot of AQI by State')
plt.xlabel('State')
plt.ylabel('AQI')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
correlation = df[['Min', 'Max', 'Avg', 'AQI']].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Between Pollutant Levels')
plt.show()

In [ ]:


plt.figure(figsize=(14, 7))
state_aqi = df.groupby('State')['AQI'].mean().sort_values(ascending=False).reset_index()

sns.barplot(x='State', y='AQI', data=state_aqi, palette='Set2')
plt.xticks(rotation=90)
plt.xlabel('State')
plt.ylabel('Average AQI')
plt.title('Average AQI by State')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(50,20))
city_aqi = df.groupby('City')['AQI'].mean().sort_values(ascending=False)
city_aqi.plot(kind='bar',color='green')
plt.title('Average AQI by City')
plt.xlabel('City')
plt.ylabel('Average AQI')
plt.show()


In [ ]:

predominant_pollutants = df.groupby(['State', 'Predominant Parameter']).size().unstack().fillna(0)
predominant_pollutants.plot(kind='bar', stacked=True, figsize=(14, 7))
plt.title('Predominant Pollutants in Different States')
plt.xlabel('State')
plt.ylabel('Count')
plt.legend(title='Pollutant')
plt.show()

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
numeric_columns = df_encoded.select_dtypes(include=['float64', 'int64']).columns
X = df_encoded.drop(columns=['AQI'])
y = df_encoded['AQI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


In [ ]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    objective='reg:squarederror')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
print('R² Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

In [ ]:
predicted = model.predict(X_test)
residuals = y_test - predicted
plt.scatter(y_test, predicted, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='black')
plt.xlabel("Actual AQI")
plt.ylabel("Predicted AQI")
plt.title("Predicted vs Actual Values")
plt.show()


In [ ]:
import requests
def fetch_realtime_aqi(city, token):
    url = f"https://api.waqi.info/feed/{city}/?token={token}"
    try:
        response = requests.get(url)
        data = response.json()

        if data["status"] == "ok":
            aqi = data["data"]["aqi"]
            dominentpol = data["data"].get("dominentpol", "unknown")
            print(f"\n✅ Real-time AQI for {city.title()}: {aqi}")
            print(f"Main pollutant: {dominentpol}")
            return aqi
        else:
            print("❌ Failed to get AQI data. Reason:", data.get("data", "Unknown error"))
            return None
    except Exception as e:
        print("❌ Error occurred while fetching AQI:", e)
        return None
token = "3c1ddc7886e2faf610c0737f375e8b4be88c1e14" # replace it with your api token
city = input("Enter city name: ").strip()
fetch_realtime_aqi(city, token)
